https://github.com/ukcp-data/ukcp-spatial-files/tree/master/spatial-files/ukcp18-uk-land-5km
https://catalogue.ceda.ac.uk/uuid/e304987739e04cdc960598fa5e4439d0/

In [1]:
import xarray as xr
import os
import glob
import pandas as pd
import geopandas as gpd
from rasterio import features
import rasterio
from shapely.geometry import shape
import os
from shapely.geometry import shape
from shapely.geometry.polygon import orient
import json
import xclim.indices
from collections import defaultdict

input_folder = "./raw/pr"
output_folder = "./processed/pr"

In [2]:
def change_from_baseline(data, output, name, period=15):
    first_period = data.isel(year=slice(0, period))
    baseline_mean = first_period.mean(dim='year')

    change_from_baseline = data - baseline_mean

    trimmed_change = change_from_baseline.isel(year=slice(15, None))

    change_path = os.path.join(output, '01_rolling', name)
    trimmed_change.to_netcdf(change_path)
# change_from_baseline(merged_ds, output_folder, "rolling_change_from_baseline.nc", period=15)

def tif_to_geojson(tif_folder, geojson_folder):
    changesYears = [35, 45, 55, 65, 75, 85]
    absYears = [50, 60, 70, 80, 90, 100]
    os.makedirs(geojson_folder, exist_ok=True)
    tif_files = glob.glob(os.path.join(tif_folder, "*.tif"))
    for tif_file in tif_files:
        years = absYears #if 'Absolute' in os.path.basename(tif_file) else changesYears
        with rasterio.open(tif_file) as src:
            # Try to get timebands from metadata (e.g., tags or band descriptions)
            timebands = []
            if src.count > 1:
                timebands = [src.descriptions[i] if src.descriptions[i] else f"{i+1}" for i in range(src.count)]
            else:
                timeband = src.tags().get('timeband', None)
                timebands = [timeband if timeband else "unknown"]

            # Only keep timebands that match the years list
            selected_indices = [i for i, tb in enumerate(timebands) if any(str(y) in str(tb) for y in years)]
            for band_idx in selected_indices:
                image = src.read(band_idx + 1)
                mask = image != src.nodata
                shapes_gen = features.shapes(image, mask=mask, transform=src.transform)
                geoms = []
                for geom, value in shapes_gen:
                    if value != src.nodata:
                        oriented_geom = orient(shape(geom), sign=1.0)
                        geoms.append({
                            'geometry': oriented_geom,
                            'properties': {'value': value, 'timeband': timebands[band_idx]}
                        })
                gdf = gpd.GeoDataFrame.from_features(geoms, crs=src.crs)
                gdf = gdf.to_crs("EPSG:4326")
                geojson_path = os.path.join(
                    geojson_folder,
                    f"{os.path.splitext(os.path.basename(tif_file))[0]}_{timebands[band_idx]}.geojson"
                )
                gdf.to_file(geojson_path, driver="GeoJSON")
                
def inverseGeojson(filename):
    file_paths = [
    os.path.join(output_folder, "03_geojson", f"{filename}_50.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_60.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_70.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_80.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_90.geojson"),
    os.path.join(output_folder, "03_geojson", f"{filename}_100.geojson"),
]

    file_labels = [
        "2030",
        "2040",
        "2050",
        "2060",
        "2070",
        "2080"
    ]

    # Dictionary to store: { coordinate: [value1, value2, ...] }
    coordinate_data = defaultdict(lambda: [None] * len(file_paths))

    # Process each file
    for idx, file_path in enumerate(file_paths):
        with open(file_path, 'r') as f:
            geojson = json.load(f)
            for feature in geojson.get("features", []):
                geometry = feature.get("geometry", {})
                properties = feature.get("properties", {})
                value = properties.get("value")  # change if your key is different

                if geometry.get("type") == "Polygon":
                    coords = geometry.get("coordinates", [])
                    if coords and coords[0]:  # outer ring exists
                        raw_coord = coords[0][0]  # first coordinate of outer ring
                        # Format coordinate to 10 decimal places, pad with zeros if necessary
                        coord_key = tuple([f"{c:.10f}" for c in raw_coord])
                        coordinate_data[coord_key][idx] = value

    # Convert to DataFrame
    df = pd.DataFrame([
        {"Coordinate": coord, **{file_labels[i]: values[i] for i in range(len(file_labels))}}
        for coord, values in coordinate_data.items()
    ])

    # Optional: remove rows where all values are None
    # df = df.dropna(subset=file_labels, how='all')

    # Show result
    df.to_json(os.path.join(output_folder, "04_inverse", f"{filename}_inverse.json"), orient='records', indent=4)
    
def graphData():
    file_paths = {
        "PR_3day": os.path.join(output_folder, "04_inverse", "PR_3day_inverse.json"),
        "PR_MaxPR":os.path.join(output_folder, "04_inverse", "PR_MaxPR_inverse.json"),
        "PR_YearlySum":os.path.join(output_folder, "04_inverse", "PR_YearlySum_inverse.json"),
    }
    
    data_by_metric = {}

    # Load each file and organize by coordinate
    for metric, path in file_paths.items():
        with open(path, 'r') as f:
            records = json.load(f)
            for entry in records:
                # Convert coordinate list to tuple for use as a dictionary key
                coord_key = tuple(entry["Coordinate"])
                if coord_key not in data_by_metric:
                    data_by_metric[coord_key] = {}
                # Extract values for 2030 to 2080 in 10-year steps
                values = [entry.get(str(year)) for year in range(2030, 2090, 10)]
                data_by_metric[coord_key][metric] = values

    # Convert coordinate tuples to strings for JSON serialization
    final_output = {str(k): v for k, v in data_by_metric.items()}

    # Save the final merged dictionary as a new JSON file
    output_file = os.path.join(output_folder, "04_inverse/graph_data.json")
    with open(output_file, 'w') as out_file:
        json.dump(final_output, out_file, indent=2)
        

In [ ]:
nc_file_list = glob.glob(os.path.join(input_folder, "*.nc"))
rolling_period = 1
if nc_file_list:
    combined_ds = xr.open_mfdataset(nc_file_list, combine='by_coords')
    # Calculate yearly metrics and aggregate into a new dataset
    yearly_groups = []
    for i, (year, yearly_ds) in enumerate(combined_ds.groupby('time.year')):
        if i >= 5:
            break
        augmented_yearly_data = yearly_ds.max(dim='time', keep_attrs=True)
        yearly_sum = yearly_ds['pr'].sum(dim='time', keep_attrs=True)
        rolling_3day_sum = yearly_ds['pr'].rolling(time=3, min_periods=3).sum()
        max_3day_sum = rolling_3day_sum.max(dim='time', keep_attrs=True)
        days_under_1mm = (yearly_ds['pr'] < 1).sum(dim='time', keep_attrs=True)
        augmented_yearly_data['days_under_1mm'] = days_under_1mm
        augmented_yearly_data['max_3day_sum'] = max_3day_sum
        augmented_yearly_data['sum'] = yearly_sum
        # Calculate SPI for each day using xclim.indices.spi
        # Use a 30-day window (or adjust as needed)
        # spi = xclim.indicators.atmos.standardized_precipitation_index(yearly_ds['pr'], freq='D', window=30)
        # augmented_yearly_data['spi'] = spi
        # augmented_yearly_data['spi-1'] = (augmented_yearly_data['spi'] < -1).sum(dim='time')
        # augmented_yearly_data['spi-2'] = (augmented_yearly_data['spi'] < -2).sum(dim='time')
        yearly_groups.append(augmented_yearly_data)
        
    # Concatenate all years along a new 'year' dimension
    merged_ds = xr.concat(yearly_groups, dim='year')
    
    combined_ds.close()
    merged_ds['rolling_sum'] = merged_ds['sum'].rolling(year=rolling_period, center=True, min_periods=1).mean()
    merged_ds['rolling_max_3day_sum'] = merged_ds['max_3day_sum'].rolling(year=rolling_period, center=True, min_periods=1).mean()
    merged_ds['rolling_pr'] = merged_ds['pr'].rolling(year=rolling_period, center=True, min_periods=1).mean()

    merged_ds = merged_ds.drop_vars(['sum', 'max_3day_sum', 'pr'])
    out_rolling_path = os.path.join(output_folder, '01_rolling', "rolling_average_04.nc")
    merged_ds.to_netcdf(out_rolling_path)

c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\core\cfchecks.py:79: UserWarning: Variable has a non-conforming standard_name: Got `lwe_precipitation_rate`, expected `['precipitation_flux']`
  check_valid(vardata, "standard_name", data["standard_name"])
c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\core\cfchecks.py:79: UserWarning: Variable has a non-conforming standard_name: Got `lwe_precipitation_rate`, expected `['precipitation_flux']`
  check_valid(vardata, "standard_name", data["standard_name"])
c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\core\cfchecks.py:79: UserWarning: Variable has a non-conforming standard_name: Got `lwe_precipitation_rate`, expected `['precipitation_flux']`
  check_valid(vardata, "standard_name", data["standard_name"])
c:\Users\RYAN.DEMELLO\Code\climate-data-viewer\.venv\Lib\site-packages\xclim\core\cfchecks.py:79: UserWarning: Variable has a non-conforming standard_n

ValueError: year already exists as coordinate or variable name.

In [6]:
yearly_groups[0]

<xarray.Dataset> Size: 2MB
Dimensions:                       (ensemble_member: 1,
                                   projection_y_coordinate: 244,
                                   projection_x_coordinate: 180, bnds: 2,
                                   time: 0)
Coordinates:
  * ensemble_member               (ensemble_member) int32 4B 4
  * projection_y_coordinate       (projection_y_coordinate) float64 2kB -3.25...
  * projection_x_coordinate       (projection_x_coordinate) float64 1kB -1.97...
    ensemble_member_id            (ensemble_member) |S27 27B dask.array<chunksize=(1,), meta=np.ndarray>
    latitude                      (projection_y_coordinate, projection_x_coordinate) float64 351kB dask.array<chunksize=(244, 180), meta=np.ndarray>
    longitude                     (projection_y_coordinate, projection_x_coordinate) float64 351kB dask.array<chunksize=(244, 180), meta=np.ndarray>
  * time                          (time) object 0B 
    prob_of_zero                  (ensemble_member, time, projection_y_coordinate, projection_x_coordinate) float64 0B dask.array<chunksize=(1, 0, 244, 180), meta=np.ndarray>
    month_number                  (time) int32 0B dask.array<chunksize=(0,), meta=np.ndarray>
    year                          (time) int32 0B dask.array<chunksize=(0,), meta=np.ndarray>
    yyyymmdd                      (time) |S64 0B dask.array<chunksize=(0,), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    pr                            (ensemble_member, projection_y_coordinate, projection_x_coordinate) float32 176kB dask.array<chunksize=(1, 244, 180), meta=np.ndarray>
    transverse_mercator           int32 4B -2147483647
    time_bnds                     (bnds) object 16B dask.array<chunksize=(2,), meta=np.ndarray>
    projection_y_coordinate_bnds  (projection_y_coordinate, bnds) float64 4kB dask.array<chunksize=(244, 2), meta=np.ndarray>
    projection_x_coordinate_bnds  (projection_x_coordinate, bnds) float64 3kB dask.array<chunksize=(180, 2), meta=np.ndarray>
    days_under_1mm                (ensemble_member, projection_y_coordinate, projection_x_coordinate) int64 351kB dask.array<chunksize=(1, 244, 180), meta=np.ndarray>
    max_3day_sum                  (ensemble_member, projection_y_coordinate, projection_x_coordinate) float32 176kB dask.array<chunksize=(1, 244, 180), meta=np.ndarray>
    sum                           (ensemble_member, projection_y_coordinate, projection_x_coordinate) float32 176kB dask.array<chunksize=(1, 244, 180), meta=np.ndarray>
    spi                           (ensemble_member, time, projection_y_coordinate, projection_x_coordinate) float64 0B dask.array<chunksize=(1, 0, 244, 180), meta=np.ndarray>
Attributes: (12/15)
    collection:      land-cpm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2021-06-09T16:58:26
    domain:          uk
    frequency:       day
    institution:     Met Office Hadley Centre (MOHC), FitzRoy Road, Exeter, D...
    ...              ...
    resolution:      5km
    scenario:        rcp85
    source:          UKCP18 realisation from a set of 12 convection-permittin...
    title:           UKCP18 land projections - Regridded 2.2km convection-per...
    version:         v20210615
    Conventions:     CF-1.7

In [7]:
combined_ds

<xarray.Dataset> Size: 7GB
Dimensions:                       (ensemble_member: 1, time: 36000,
                                   projection_y_coordinate: 244,
                                   projection_x_coordinate: 180, bnds: 2)
Coordinates:
  * ensemble_member               (ensemble_member) int32 4B 4
  * time                          (time) object 288kB 1980-12-01 12:00:00 ......
  * projection_y_coordinate       (projection_y_coordinate) float64 2kB -3.25...
  * projection_x_coordinate       (projection_x_coordinate) float64 1kB -1.97...
    ensemble_member_id            (ensemble_member) |S27 27B dask.array<chunksize=(1,), meta=np.ndarray>
    latitude                      (projection_y_coordinate, projection_x_coordinate) float64 351kB dask.array<chunksize=(244, 180), meta=np.ndarray>
    longitude                     (projection_y_coordinate, projection_x_coordinate) float64 351kB dask.array<chunksize=(244, 180), meta=np.ndarray>
    month_number                  (time) int32 144kB dask.array<chunksize=(3600,), meta=np.ndarray>
    year                          (time) int32 144kB dask.array<chunksize=(3600,), meta=np.ndarray>
    yyyymmdd                      (time) |S64 2MB dask.array<chunksize=(3600,), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    pr                            (ensemble_member, time, projection_y_coordinate, projection_x_coordinate) float32 6GB dask.array<chunksize=(1, 3600, 244, 180), meta=np.ndarray>
    transverse_mercator           (time) int32 144kB -2147483647 ... -2147483647
    time_bnds                     (time, bnds) object 576kB dask.array<chunksize=(3600, 2), meta=np.ndarray>
    projection_y_coordinate_bnds  (time, projection_y_coordinate, bnds) float64 141MB dask.array<chunksize=(3600, 244, 2), meta=np.ndarray>
    projection_x_coordinate_bnds  (time, projection_x_coordinate, bnds) float64 104MB dask.array<chunksize=(3600, 180, 2), meta=np.ndarray>
Attributes: (12/15)
    collection:      land-cpm
    contact:         ukcpproject@metoffice.gov.uk
    creation_date:   2021-06-09T16:58:26
    domain:          uk
    frequency:       day
    institution:     Met Office Hadley Centre (MOHC), FitzRoy Road, Exeter, D...
    ...              ...
    resolution:      5km
    scenario:        rcp85
    source:          UKCP18 realisation from a set of 12 convection-permittin...
    title:           UKCP18 land projections - Regridded 2.2km convection-per...
    version:         v20210615
    Conventions:     CF-1.7

In [ ]:
tif_to_geojson(os.path.join(output_folder, "02_clipped"), os.path.join(output_folder, "03_geojson"))

In [ ]:
inverseGeojson("PR_3day")
inverseGeojson("PR_MaxPR")
inverseGeojson("PR_YearlySum")

In [ ]:
graphData()